In [5]:
from generate_probs import Simulator
import plotly.express as px
import numpy as np
import pandas as pd

In [2]:
success_criteria =  {(2, 0): 1} #, (4, 1): 3, (4, 2): 3, (4, 3): 3}
copies_already_held =  {(2, 0): 2}
champions_missing = {2: 20} # {1: 40, 2: 20, 3: 10}
level = 6
N = 100000

sim = Simulator(success_criteria, level=level, copies_already_held=copies_already_held, champions_missing=champions_missing, semantics='or')
res = sim.simulate_dist(success_criteria, n=N)

{1: [29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29], 2: [20, 2, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22], 3: [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18], 4: [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11], 5: [10, 10, 10, 10, 10, 10, 10]}


100%|██████████| 100000/100000 [02:54<00:00, 573.05it/s]


In [14]:
import numpy as np
import pandas as pd
from uuid import uuid4
from generate_probs import Simulator
import json

def serialize_param_dict(params):
    new_params = {}
    for key, value in params.items():
        if type(value) == dict: # serialize tuples to string manually, not handled by json.dumps
            new_params[key] = {str(k): v for k,v in value.items()}
        else:
            new_params[key] = value
    return json.dumps(new_params)

def params_exist(params, idx_df):
    param_string = serialize_param_dict(params)
    return len(idx_df[idx_df['params'] == param_string]) > 0 

def simulate_and_save(params, idx_directory):
    try:
        idx_df = pd.read_csv(idx_directory)
    except:
        idx_df = pd.DataFrame(columns=['params', 'filename'])
    if params_exist(params, idx_df):
        print('parameters already exist!')
        return
    new_id = uuid4()
    file_name = f'../distributions/{new_id}.npy'
    print(f'simulating distrubtion for the following parameters')
    sim = Simulator(params['success_criteria'], 
                    level=params['level'], 
                    copies_already_held=params['copies_already_held'], 
                    champions_missing=params['champions_missing'], 
                    semantics=params['semantics'])
    dist = sim.simulate_dist(params['success_criteria'], n=params['N'])
    np.save(file_name, dist)
    idx_df.loc[len(idx_df)] = [serialize_param_dict(params), file_name]
    idx_df.to_csv(idx_directory, index=False)


params = {
    'success_criteria':  {(2, 0): 1},
    'copies_already_held':  {(2, 0): 2},
    'champions_missing': {2: 20},
    'level': 6,
    'N': 5,
    'semantics': 'or'
}

simulate_and_save(params, './params_mapping.csv')

parameters already exist!


In [ ]:
# level 4 situations

for copies_held in range(9):
    params = {
        'success_criteria':  {(1, 0): 9 - copies_held},
        'copies_already_held':  {(2, 0): copies_held},
        'champions_missing': {1: 20}, # roughly each person in the lobby has a 2 star 1 cost.
        'level': 4,
        'N': 100000,
        'semantics': 'or'
    }

    simulate_and_save(params, './params_mapping.csv')

In [18]:

# make slicing soon
def load_npy_files(idx_df):
    idx_df['dists'] = idx_df['filename'].map(lambda filename: np.load(filename))
    return idx_df

idx_df = pd.read_csv('params_mapping.csv')
idx_df = load_npy_files(idx_df)

In [31]:
dist = idx_df['dists'][7]

print(np.mean(dist), np.median(dist), np.percentile(dist, 90))
px.histogram(dist)

10.97261 9.0 20.0


In [3]:
px.histogram(res)

In [4]:
np.mean(res), np.median(res), np.percentile(res, 90)

(7.92082, 6.0, 16.0)

In [4]:
px.histogram(res)

In [8]:
import numpy as np

np.mean(res), np.median(res), np.percentile(res, 90)

(34.8503, 29.0, 66.0)